In [1]:
pip install -U sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=31950e0edac394ac3d964630c9c31c2444ceb4a50f8b140dc4f864cf2cea5f11
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
from sentence_transformers import SentenceTransformer, util
import json
import pandas as pd
import numpy as np
from collections import Counter

In [75]:
#model  = SentenceTransformer('bert-base-multilingual-cased') #0.7/0.69
model  = SentenceTransformer('distiluse-base-multilingual-cased-v2') #0.76/0.75

In [76]:
with open("processed_binary_df.json", "r") as f:
    trial_data = json.load(f)


df = pd.json_normalize(trial_data)

#df.head(79)

,hyp,src,tgt,task,word_to_define,is_hallucination,named_entities,strong_hallu,full_hallu
0,resembling or characteristic of a weasel,The writer had just entered into his eighteent...,resemble a weasel in appearance,DM,weaselly,0.0,0.0,0.0,0.0
1,alternative form of sheath knife,Sailors ' and fishermen 's <define> sheath - k...,,DM,sheath - knives,1.0,0.0,1.0,0.0
2,obsolete a short period of time,"As to age , Bead could not form any clear impr...",poetic an instant a short moment,DM,eyewink,0.0,0.0,0.0,0.0
3,slang an incel,Because redpillers are usually normies or <def...,incel slang a man of a slightly lower rank on ...,DM,Chadlites,0.0,0.0,0.0,0.0
4,an island in Lienchiang County Taiwan,On the second day of massive live - fire drill...,an island in Dongyin Lienchiang Taiwan in the ...,DM,Xiyin,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
74,intransitive slang to have sex,Pimpin � came so naturally to MT when he and h...,US slang,DM,hoe,1.0,0.0,1.0,0.0
75,transitive slang to fail an examination,"I met one of your dons at tea , and he said th...",agitated confused,DM,flurried,1.0,0.0,0.0,1.0
76,figuratively of a person have a tendency to st...,"And on tonight �s episode , when she faces a c...",prone to stumble,DM,stumbly,1.0,0.0,1.0,0.0
77,not respond,Replying students did not differ significantly...,not reply to a survey etc,DM,nonreplying,0.0,0.0,0.0,0.0


In [77]:
result_dict = {}
hallucination_cosines_total = 0
hallucinations = []
hallucinations_cosines=[]
cosine_similarities = []

for i in range(0, len(df)):
    sentences = [df.iloc[i]['hyp'], df.iloc[i]['tgt']]
    embedding = model.encode(sentences, convert_to_tensor=False) #tokenization is implicitly handled by the SentenceTransformer here
    cosine_scores = util.cos_sim(embedding, embedding)

    for id_h, h in enumerate(sentences):
        for id_t, t in enumerate(sentences):
            if id_h >= id_t:
                continue
            dp_id = len(result_dict)  #Using dp_id as the key
            #if df.iloc[i]['p(Hallucination)'] >= 0.66:
            if df.iloc[i]['is_hallucination'] == 1:
                hallucinations.append(dp_id)
                hallucination_cosines_total += cosine_scores[id_h][id_t].item()
                hallucinations_cosines.append(cosine_scores[id_h][id_t].item())

            result_dict[dp_id] = {'hyp': h, 'tgt': t, 'cosine_similarity': cosine_scores[id_h][id_t].item()}

for dp_id, values in result_dict.items():
    cosine_similarities.append(values['cosine_similarity'])
    print(f'Datapoint {dp_id}:')
    print(f'hyp: {values["hyp"]}')
    print(f'tgt: {values["tgt"]}')
    print(f'cosine similarity: {values["cosine_similarity"]}')
    print()


Datapoint 0:
hyp: resembling or characteristic of a weasel
tgt: resemble a weasel in appearance
cosine similarity: 0.6999791860580444

Datapoint 1:
hyp: alternative form of sheath knife
tgt: 
cosine similarity: 0.06260668486356735

Datapoint 2:
hyp: obsolete a short period of time
tgt: poetic an instant a short moment
cosine similarity: 0.6154297590255737

Datapoint 3:
hyp: slang an incel
tgt: incel slang a man of a slightly lower rank on a scale of physical attractiveness than Chad
cosine similarity: 0.1903480887413025

Datapoint 4:
hyp: an island in Lienchiang County Taiwan
tgt: an island in Dongyin Lienchiang Taiwan in the Matsu Islands
cosine similarity: 0.8584123849868774

Datapoint 5:
hyp: alternative form of blue bearded
tgt: have thick dark facial hair
cosine similarity: 0.39380431175231934

Datapoint 6:
hyp: baseball a sacrifice bunt
tgt: baseball a ball that have be intentionally hit softly with a hand spread batting stance with the intention of advance a runner at the cost o

In [78]:
threshold = np.mean(hallucinations_cosines)

hallucinations_pred = []

for dp_id, values in result_dict.items():
    similarity = values['cosine_similarity']
    if similarity < threshold:
        hallucinations_pred.append(dp_id)

counter1 = Counter(hallucinations)
counter2 = Counter(hallucinations_pred)

intersection = counter1 & counter2

total_mutual_elements = sum(intersection.values())

print(f'Threshold: {threshold}')
print(f'Total Hallucinations: {len(hallucinations)}')
print(f'Total Hallucinations Predicted: {len(hallucinations_pred)}')
print('Percentage of correctly predicted hallucinations:' , (total_mutual_elements / len(hallucinations)) * 100 , '%')
print('Correct Hallucinations:', intersection.keys())
print('Number of correct Hallucinations:', len(intersection.keys()))
print(hallucinations)

Threshold: 0.5239502628709813
Total Hallucinations: 218
Total Hallucinations Predicted: 150
Percentage of correctly predicted hallucinations: 50.0 %
Correct Hallucinations: dict_keys([1, 5, 7, 9, 13, 16, 17, 19, 20, 32, 33, 34, 36, 42, 45, 46, 47, 48, 49, 51, 54, 59, 60, 61, 62, 63, 65, 66, 69, 72, 74, 75, 76, 85, 90, 92, 93, 98, 100, 105, 106, 109, 115, 118, 122, 125, 126, 131, 134, 135, 139, 140, 142, 144, 146, 147, 153, 155, 157, 160, 162, 170, 173, 174, 175, 176, 177, 181, 182, 184, 186, 188, 199, 212, 216, 221, 223, 225, 228, 230, 235, 242, 252, 260, 285, 290, 293, 299, 314, 319, 321, 322, 326, 333, 337, 338, 340, 341, 350, 353, 354, 355, 356, 358, 359, 360, 361, 377, 400])
Number of correct Hallucinations: 109
[1, 5, 7, 9, 11, 12, 13, 14, 16, 17, 19, 20, 32, 33, 34, 35, 36, 38, 41, 42, 45, 46, 47, 48, 49, 51, 54, 59, 60, 61, 62, 63, 65, 66, 69, 72, 74, 75, 76, 85, 90, 92, 93, 98, 100, 102, 104, 105, 106, 108, 109, 112, 113, 114, 115, 118, 122, 123, 125, 126, 128, 131, 134, 135, 1

In [79]:
all_metrics = pd.concat([
    pd.DataFrame({
        'cos_sim': -np.array(cosine_similarities),
    })], axis=1)

In [80]:
from sklearn.metrics import roc_auc_score

In [81]:
target_columns = ['is_hallucination'	, 'full_hallu',	'strong_hallu']

In [82]:
aucs = pd.DataFrame({
    target: {pred: roc_auc_score(df[target], all_metrics[pred]) for pred in all_metrics.columns}
    for target in target_columns
})
aucs.sort_values('is_hallucination', ascending=False)

,is_hallucination,full_hallu,strong_hallu
cos_sim,0.762921,0.721513,0.715851
